<a href="https://colab.research.google.com/github/noemigarcia27/SImulacion-II/blob/main/Reduccion_de_la_varianza_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reducción de la varianza**
# Variables antitéticas

Las librerias que vamos a ocupar son las siguientes

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random as rd
import time

Nuestra función para este ejercicio es:
$$ g(x) = \sqrt{arctan(x)} dx  $$

In [3]:
def g(x):
  return np.sqrt(np.arctan(x))

El siguiente codigo es el método de **Monte Carlo crudo** que ya habiamos hecho anteriormente

In [4]:
def mc_crudo(N, n, func=g, a=0.0, b=1.0):
  thetas = []
  t0 = time.perf_counter()
  for _ in range(n):
      suma = 0.0
      for _ in range(N):
        u = rd.random()
        x = a + u * (b - a)
        suma += func(x)
      theta2 = (b - a) / N * suma
      thetas.append(theta2)
  t1 = time.perf_counter()
  tiempo_promedio = (t1 - t0) / n
  return np.mean(thetas), np.var(thetas), tiempo_promedio

Vamos a hacer la reducción con el método de variables antitéticas para el método de Monte Carlo crudo, usando el teorema
$$ \text{Var} ( \frac{ X_1 + X_2 }{2} ) = \frac{1}{4} [ \text{Var} (X_1) + \text{Var} (X_2) + 2 \text{Cov} ( X_1 + X_2 )] $$



En el siguiente codigo en vez de usar muestras u (como en el Método MC crudo) se utiliza 1-u

In [5]:
def mc_crudo_1u(N, n, func=g, a=0.0, b=1.0):
  thetas = []
  t0 = time.perf_counter()
  for _ in range(n):
      suma = 0.0
      for _ in range(N):
        u = rd.random()
        x = a + (1-u) * (b - a)
        suma += func(x)
      theta2 = (b - a) / N * suma
      thetas.append(theta2)
  t1 = time.perf_counter()
  tiempo_promedio = (t1 - t0) / n
  return np.mean(thetas), np.var(thetas), tiempo_promedio

En el siguiente lo que hacermos es usar ambos y sacamos el promedio es decir, es lo que hace realmente la reduccion de la varianza
$$ \text{Var} ( \frac{ f(u) + f(1-u) }{2} ) < \text{Var} (f(u)) $$

In [8]:
def mc_crudo_antiteticas(N, n, func=g, a=0.0, b=1.0):
  thetas = []
  t0 = time.perf_counter()
  for i in range(n):
    suma = 0.0
    for j in range(N):
      u = rd.random()
      x1 = a + u * (b - a)
      x2 = a + (1.0 - u) * (b - a)
      suma += ((func(x1) + func(x2)))/2 #promedio de g(u) y g(1-u)
    theta = (b - a) * (suma / N)
    thetas.append(theta)
  t1 = time.perf_counter()
  tiempo_promedio = (t1 - t0) / n
  return np.mean(thetas), np.var(thetas), tiempo_promedio

In [9]:
N=1000
n=30

mean_c, var_c, t_c = mc_crudo(N, n)
mean_1u, var_1u, t_1u = mc_crudo_1u(N, n)
mean_ant, var_ant, t_ant = mc_crudo_antiteticas(N, n)

In [10]:
print("Monte Carlo crudo con u:               media =", mean_c, " varianza =", var_c, " tiempo =", t_c)
print("Monte Carlo crudo con 1-u:             media =", mean_1u, " varianza =", var_1u, " tiempo =", t_1u)
print("Monte Carlo crudo con la red de var:   media =", mean_ant, " varianza =", var_ant, " tiempo =", t_ant)

Monte Carlo crudo con u:               media = 0.6308860959754247  varianza = 4.3490910294762026e-05  tiempo = 0.002342864133333933
Monte Carlo crudo con 1-u:             media = 0.6311539641713417  varianza = 2.92588272550546e-05  tiempo = 0.002716570466676179
Monte Carlo crudo con la red de var:   media = 0.6293376970436785  varianza = 2.5087377564140458e-06  tiempo = 0.004143674466664985


Para ver en cuanto se redujo la varianza ocupamos lo siguiente
$$ \text{Reduccion} = \frac{\text{Varianza Original - Varianza Nueva}}{\text{Varianza Original}} * 100 $$

In [12]:
reduccion=((var_c-var_ant)/var_c)*100
print("Reduccion de la varianza: ", reduccion, "%")

Reduccion de la varianza:  94.23158140537657 %


Ahora, para el método de **Monte Carlo acierto y error** podemos hacerlo con antiteticas con pares, es decir $ (u,v) $ y $ (1-u, 1-v) $  

Esto funciona porque los dos indicaadores estan negativamente correlacionados
$$ \text{Var}(\tilde{I}) = \frac{ \text{Var}(\tilde{Z}) }{ N/2 } = \frac{1}{N/2} \cdot \frac{1}{4} [ 2 \text{Var}(Z) + 2 \text{Cov}(Z_1,Z_2) ] $$
Si $ \text{Cov}(Z_1,Z_2) < 0 $, entonces

$$Var(\tilde{I})<Var(\hat{I})$$

donde $ \tilde{I} = \frac{1}{N/2} \sum_{k=1}^{N/2} \tilde{Z}_k, \hat{I} = \frac{1}{N} \sum_{i=1}^{N} Z_i, \tilde{Z} = \frac{Z_1 + Z_2}{2} $

In [13]:
def mc_acierto_antiteticas(N, n, func=g, a=0.0, b=1.0, c=1.0):
  thetas = []
  t0 = time.perf_counter()
  for _ in range(n):  # n estimaciones
    aciertos = 0
    for _ in range(N // 2):  # N/2 pares antitéticos
      u1, v1 = rd.random(), rd.random()
      u2, v2 = 1 - u1, 1 - v1
      x1 = a + u1 * (b - a)
      x2 = a + u2 * (b - a)
      # Contamos los aciertos para ambos pares
      aciertos += int(v1 <= func(x1) / c)
      aciertos += int(v2 <= func(x2) / c)
    theta = (b - a) * c * (aciertos / N)
    thetas.append(theta)
  t1 = time.perf_counter()
  tiempo_promedio = (t1 - t0) / n
  return np.mean(thetas), np.var(thetas), tiempo_promedio

El codigo inicial era el siguiente:

In [16]:
def mc_acierto(N, n, func=g, a=0.0, b=1.0, c=1.0):
  thetas = []
  t0 = time.perf_counter()
  for _ in range(n):              # n repeticiones
    Na = 0
    for _ in range(N):          # N puntos en cada corrida
      x = rd.random()*(b-a)+a # x uniformemente en [a,b]
      y = rd.random()*c       # y uniformemente en [0,c]
      if y <= func(x):
        Na += 1
    theta1 = c*(b-a)*(Na/N)     # Valor de \theta_1
    thetas.append(theta1)
  t1 = time.perf_counter()
  tiempo_promedio = (t1 - t0) / n
  return np.mean(thetas), np.var(thetas), tiempo_promedio

In [17]:
N=1000
n=30

mean_acierto, var_acierto, t_acierto = mc_acierto(N,n)
mean_red, var_red, t_red = mc_acierto_antiteticas(N,n)

In [20]:
print("Monte Carlo acierto y error:                 media", mean_acierto, "varianza", var_acierto, "tiempo", t_acierto)
print("Monte Carlo acierto y error con antiteticas: media", mean_red, "varianza", var_red, "tiempo", t_red)

Monte Carlo acierto y error:                 media 0.6302333333333333 varianza 0.00018711222222222252 tiempo 0.002335390099991249
Monte Carlo acierto y error con antiteticas: media 0.6298333333333334 varianza 9.087222222222239e-05 tiempo 0.0026683792999998937


Para ver en cuanto se redujo la varianza ocupamos lo siguiente
$$ \text{Reduccion} = \frac{\text{Varianza Original - Varianza Nueva}}{\text{Varianza Original}} * 100 $$

In [22]:
reduccion=((var_acierto-var_red)/var_acierto)*100
print("Reduccion de la varianza: ", reduccion, "%")

Reduccion de la varianza:  51.43437390514307 %
